In [32]:
import pickle
import pandas as pd
import os

In [46]:
pickle_name = "./Results/Run38/Results_final_pickle.pkl"
# pickle_name = "./Results/Run23/Results29_pickle.pkl"

with open(pickle_name, 'rb') as handle:
    dict_data = pickle.load(handle)
df = pd.DataFrame.from_dict(dict_data)

In [47]:
df.sort_values(['Performance'], ascending=[False],inplace=True)
df['ratio n1/b1'] = df['n1'] /df['b1'] 
df['ratio n2/b2'] = df['n2'] /df['b2'] 
df['ratio n3/b3'] = df['n3'] /df['b3'] 

df

,Epoch,Ant,Performance,init,n1,n2,n3,simd,Olevel,num_thread,b1,b2,b3
559,14,0,1166.56,init,512,512,512,avx2,-O3,16,272,16,64
199,5,0,1163.60,init,512,512,512,avx2,-O2,16,272,16,64
359,9,0,1163.06,init,512,512,512,avx2,-Ofast,16,272,16,64
239,6,0,1162.03,init,512,512,512,avx2,-O3,16,272,16,112
200,5,1,1144.59,init,512,512,512,avx2,-Ofast,16,272,16,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,6,39,322.79,init,512,512,512,avx512,-O3,16,32,432,160
38,0,38,316.84,init,512,512,512,avx2,-O2,16,64,352,64
158,3,39,314.66,init,512,512,512,avx2,-Ofast,16,112,432,32
77,1,38,314.40,init,512,512,512,avx512,-O3,16,64,448,80


In [48]:
df_stat = df[['Epoch', 'Performance']].groupby(['Epoch']).mean()
df_stat

In [49]:
df.plot.scatter(x="station_london", y="station_paris")

,Performance
Epoch,
0,544.844103
1,514.189000
2,525.995500
3,507.698500
4,517.719250
5,535.363500
6,530.324750
7,506.868750
8,554.780000
